In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import shotchartleaguewide
from IPython.core.display import HTML

pd.options.mode.chained_assignment = None

shots_DIR = "../data/ShotLocationData/"
box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"

In [ ]:
season = 2024
all_shots = pd.read_parquet(shots_DIR + f"NBA_Shot_Details_{season}" + ".parquet")
all_shots.columns = map(str.lower, all_shots.columns)
all_shots = all_shots.rename(columns={"shot_made_flag":"shot_made"})
all_shots["loc_x"] = all_shots["loc_x"]/10
all_shots["loc_y"] = all_shots["loc_y"]/10
all_shots['fg2'] = np.where(all_shots['shot_type'] == '2PT Field Goal', 1, 0)
all_shots['fg3'] = np.where(all_shots['shot_type'] == '3PT Field Goal', 1, 0)
all_shots["points"] = (2*all_shots['fg2'] + 3*all_shots['fg3'])*all_shots["shot_made"]
# shots = all_shots.loc[all_shots['shot_distance'] <= 32].reset_index(drop=True)
# shots = all_shots.copy()

In [ ]:
sts = ["Dunk","Floater","Layup","Alley_Oop","Reverse","Step_Back","Hook","Turnaround","Bank","Jump","Cut","Drive","Finger_Roll","Pullup","Fadeaway","Tip"]
sts_st = ["Dunk","Floater","Layup","Alley","Reverse","Step Back","Hook","Turnaround","Bank","Jump","Cutting","Driving","Finger Roll","Pullup","Fadeaway","Tip"]
for s,st in zip(sts,sts_st):
    all_shots[s]=all_shots["action_type"].str.contains(st)
    if sts == "Pullup":
        all_shots[s]=all_shots["action_type"].str.contains(st) | all_shots["action_type"].str.contains("Pull-Up")

In [ ]:
df = all_shots.query("shot_made == True")

In [ ]:
all_shots

In [ ]:
df1 = df.groupby(["player_id","player_name"])[sts].sum().reset_index()
df1["P"]=img_DIR_P + df1["player_id"].astype(str) + ".png"

In [ ]:
cols = ["PLAYER_ID","AGE"]
df2 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Base_{season}.parquet",columns=cols)
df2.columns = map(str.lower,df2.columns)
df3 = pd.merge(df1,df2,on=["player_id"])

In [ ]:
df4 = df3.query("age >= 35").sort_values("Dunk",ascending=False).reset_index(drop=True)


In [ ]:
df4

In [ ]:
data = df4.head(10)
players = data["player_name"].to_list()
players.reverse()
data["player_name"] = pd.Categorical(data["player_name"],categories=players,ordered=True)
p= (
    ggplot(data)
    + aes(x="player_name",y="Dunk",image="P")
    + geom_col(alpha=0.8)
    + geom_image(aes(y=-0.2),size=0.15)
    + coord_flip()
    + theme_idv
    + theme(
        axis_text_y=element_blank(),
        axis_title_y=element_blank(),
        figure_size=(5,6)
    )
    + labs(
        title="Most Dunks by Players aged 35 years or older",
        subtitle=f"{get_ss(season)} NBA Season",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
)
p

In [ ]:
df4 = df3.sort_values("Dunk",ascending=False).reset_index(drop=True)
data = df4.head(10)
players = data["player_name"].to_list()
players.reverse()
data["player_name"] = pd.Categorical(data["player_name"],categories=players,ordered=True)
p= (
    ggplot(data)
    + aes(x="player_name",y="Dunk",image="P")
    + geom_col(alpha=0.8)
    + geom_image(aes(y=-0.2),size=0.15)
    + coord_flip()
    + theme_idv
    + theme(
        axis_text_y=element_blank(),
        axis_title_y=element_blank(),
        figure_size=(5,6)
    )
    + labs(
        title="Most Dunks",
        subtitle=f"{get_ss(season)} NBA Season",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
)
p